In [8]:
# %% [markdown]
# # Steganography Detection using CNN in PyTorch
# Dieses Notebook zielt darauf ab, ein Convolutional Neural Network (CNN) zu trainieren, um PNG-Bilder mit verstecktem Inhalt (z. B. JavaScript, PowerShell-Skripte und URLs) von normalen PNGs zu unterscheiden.
# Dataset: 8,000 PNGs (512x512), die entweder „clean“ oder „kompromittiert“ sind (LSB-Steganographie).

# %% [markdown]
# ## 1. Setup
# Importiere die benötigten Bibliotheken und definiere die grundlegenden Parameter.

# %%
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt

# Parameter
BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
IMG_SIZE = (512, 512)

# Device-Erkennung (GPU oder CPU) für cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# %% [markdown]
# ## 2. Datensatz vorbereiten
# Lade die Bilder und erstelle eine PyTorch Dataset-Klasse für den Umgang mit PNG-Dateien.

# %%
class SteganographyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        
        # Lade Bildpfade und Labels (0 = clean, 1 = kompromittiert)
        for label, subdir in enumerate(['clean', 'stego']):
            path = os.path.join(root_dir, subdir)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                self.data.append((img_path, label))
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformationen für die Bilder
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()
])

# Datensätze laden
train_dataset = SteganographyDataset(root_dir='data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = SteganographyDataset(root_dir='data/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# %% [markdown]
# ## 3. Modell definieren
# Erstellen eines CNN-Modells zur Unterscheidung von „clean“ und „kompromittierten“ Bildern.

# %%
class SteganoCNN(nn.Module):
    def __init__(self):
        super(SteganoCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 2)  # Binary classification

    def forward(self, x):
        return self.model(x)

model = SteganoCNN().to(device) #/to(device) ist für cuda
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# %% [markdown]
# ## 4. Training-Funktion

# %%
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    for images, labels in loader:
        #für cuda
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Training Loss: {running_loss / len(loader):.4f}, Accuracy: {accuracy:.2f}%')
    return running_loss / len(loader), accuracy

# %% [markdown]
# ## 5. Test-Funktion

# %%
def test(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            #für cuda
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Loss: {running_loss / len(loader):.4f}, Accuracy: {accuracy:.2f}%')
    return running_loss / len(loader), accuracy

# %% [markdown]
# ## 6. Training und Evaluation

# %%
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

for epoch in range(NUM_EPOCHS):
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}')
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    test_loss, test_accuracy = test(model, test_loader, criterion)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

# %% [markdown]
# ## 7. Ergebnisse visualisieren

# %%
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(NUM_EPOCHS), train_losses, label='Train Loss')
plt.plot(range(NUM_EPOCHS), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(NUM_EPOCHS), train_accuracies, label='Train Accuracy')
plt.plot(range(NUM_EPOCHS), test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

# %% [markdown]
# ## 8. Modell speichern

# %%
torch.save(model.state_dict(), 'stegano_cnn.pth')
print("Modell gespeichert als 'stegano_cnn.pth'")

/Users/flaviohorak/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/flaviohorak/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/flaviohorak/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


Epoch 1/20
